In [1]:
import pandas as pd
import os
import numpy as np

In [43]:
a = pd.read_csv("../data/01.csv")

In [32]:
df_list = []
for file in os.listdir("../data/"):
    if file[0] in ['0', '1'] and file != "01.csv":
        print("reading " + file)
        temp = pd.read_csv("../data/" + file, 
                                   usecols=list(range(48)), 
                                   encoding="latin-1")
        prev_season = str(int(file[0:2]) - 1)
        if len(prev_season) == 1:
            prev_season = '0' + prev_season
        
        temp.loc[:, "season"] = prev_season + file[0:2]
        df_list.append(temp)
df = pd.concat(df_list)

reading 06.csv
reading 14.csv
reading 17.csv
reading 13.csv
reading 03.csv
reading 11.csv
reading 18.csv
reading 05.csv
reading 10.csv
reading 09.csv
reading 08.csv
reading 19.csv
reading 15.csv
reading 07.csv
reading 04.csv
reading 02.csv
reading 12.csv
reading 16.csv


In [33]:
len(df)

6521

In [34]:
df.dropna(inplace=True, subset=["HomeTeam"])

In [35]:
len(df)

6520

In [36]:
df.to_csv("../data/pl_to_20180924.csv")

Testing

In [5]:
df = pd.read_csv("../data/pl_to_20180924.csv", index_col=0)

In [7]:
df.columns

Index(['ABP', 'AC', 'AF', 'AHW', 'AO', 'AR', 'AS', 'AST', 'AY', 'Attendance',
       'AwayTeam', 'B365<2.5', 'B365>2.5', 'B365A', 'B365D', 'B365H', 'BWA',
       'BWD', 'BWH', 'Bb1X2', 'BbAvH', 'BbMxD', 'BbMxH', 'Date', 'Div', 'FTAG',
       'FTHG', 'FTR', 'GB<2.5', 'GB>2.5', 'GBA', 'GBD', 'GBH', 'HBP', 'HC',
       'HF', 'HHW', 'HO', 'HR', 'HS', 'HST', 'HTAG', 'HTHG', 'HTR', 'HY',
       'HomeTeam', 'IWA', 'IWD', 'IWH', 'LBA', 'LBD', 'LBH', 'PSA', 'PSD',
       'PSH', 'Referee', 'SBA', 'SBD', 'SBH', 'SJA', 'SJD', 'SJH', 'SOA',
       'SOD', 'SOH', 'SYA', 'SYD', 'SYH', 'VCA', 'VCD', 'VCH', 'WHA', 'WHD',
       'WHH'],
      dtype='object')

Check columns of interest are not null

In [10]:
for column in ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 
               'FTR', 'HS', 'AS', 'HST', 'AST', 'BbMxD', 'BbMxH']:
    nulls = len(df[df[column].isnull()])
    print("For column {} there are {} nulls".format(column, nulls))
    

For column Date there are 0 nulls
For column HomeTeam there are 0 nulls
For column AwayTeam there are 0 nulls
For column FTHG there are 0 nulls
For column FTAG there are 0 nulls
For column FTR there are 0 nulls
For column HS there are 0 nulls
For column AS there are 0 nulls
For column HST there are 0 nulls
For column AST there are 0 nulls
For column BbMxD there are 5320 nulls
For column BbMxH there are 5320 nulls


Bet brain Max Draw and Home have lots of missing data. Bet brain Max away doesn't seem to exist

Iterate through all files to check where we are missing data for these

In [42]:
df_list = []
for file in sorted(os.listdir("../data/")):
    if file[0] in ['1'] and file != "01.csv":
        print("reading " + file)
        df = pd.read_csv("../data/" + file, encoding="latin-1")
        print("Is BbMxD in? {}".format('BbMxD' in df.columns))
        print("Is BbMxA in? {}".format('BbMxA' in df.columns))
        print("Is BbMxH in? {}".format('BbMxH' in df.columns))

reading 10.csv
Is BbMxD in? True
Is BbMxA in? True
Is BbMxH in? True
reading 11.csv
Is BbMxD in? True
Is BbMxA in? True
Is BbMxH in? True
reading 12.csv
Is BbMxD in? True
Is BbMxA in? True
Is BbMxH in? True
reading 13.csv
Is BbMxD in? True
Is BbMxA in? True
Is BbMxH in? True
reading 14.csv
Is BbMxD in? True
Is BbMxA in? True
Is BbMxH in? True
reading 15.csv
Is BbMxD in? True
Is BbMxA in? True
Is BbMxH in? True
reading 16.csv
Is BbMxD in? True
Is BbMxA in? True
Is BbMxH in? True
reading 17.csv
Is BbMxD in? True
Is BbMxA in? True
Is BbMxH in? True
reading 18.csv
Is BbMxD in? True
Is BbMxA in? True
Is BbMxH in? True
reading 19.csv
Is BbMxD in? True
Is BbMxA in? True
Is BbMxH in? True


OK, so let's first of all restrict ourselves to making a model just on the goals data, and then worry about betting afterwards, since we have all of the betting data here for the four years to test on

Check team names are consistent

In [23]:
df = pd.read_csv("../data/pl_to_20180924.csv", index_col=0)

In [24]:
sorted(df.HomeTeam.unique())

['Arsenal',
 'Aston Villa',
 'Birmingham',
 'Blackburn',
 'Blackpool',
 'Bolton',
 'Bournemouth',
 'Brighton',
 'Burnley',
 'Cardiff',
 'Charlton',
 'Chelsea',
 'Crystal Palace',
 'Derby',
 'Everton',
 'Fulham',
 'Huddersfield',
 'Hull',
 'Ipswich',
 'Leeds',
 'Leicester',
 'Liverpool',
 'Man City',
 'Man United',
 'Middlesboro',
 'Middlesbrough',
 'Newcastle',
 'Norwich',
 'Portsmouth',
 'QPR',
 'Reading',
 'Sheffield United',
 'Southampton',
 'Stoke',
 'Sunderland',
 'Swansea',
 'Tottenham',
 'Watford',
 'West Brom',
 'West Ham',
 'Wigan',
 'Wolves']

In [25]:
sorted(df.AwayTeam.unique())

['Arsenal',
 'Aston Villa',
 'Birmingham',
 'Blackburn',
 'Blackpool',
 'Bolton',
 'Bournemouth',
 'Brighton',
 'Burnley',
 'Cardiff',
 'Charlton',
 'Chelsea',
 'Crystal Palace',
 'Derby',
 'Everton',
 'Fulham',
 'Huddersfield',
 'Hull',
 'Ipswich',
 'Leeds',
 'Leicester',
 'Liverpool',
 'Man City',
 'Man United',
 'Middlesboro',
 'Middlesbrough',
 'Newcastle',
 'Norwich',
 'Portsmouth',
 'QPR',
 'Reading',
 'Sheffield United',
 'Southampton',
 'Stoke',
 'Sunderland',
 'Swansea',
 'Tottenham',
 'Watford',
 'West Brom',
 'West Ham',
 'Wigan',
 'Wolves']

No one seems to be repeated with different spellings

In [27]:
len(df.HomeTeam.unique()) == len(df.AwayTeam.unique())

True

In [28]:
np.setdiff1d(df.HomeTeam.unique(), df.AwayTeam.unique())

array([], dtype=object)

In [29]:
np.setdiff1d(df.AwayTeam.unique(), df.HomeTeam.unique())

array([], dtype=object)

data appears clean for all intents and purposes for building a model!

In [4]:
df_list = []
for file in sorted(os.listdir("../data/")):
    print(file)
    file = pd.read_csv("../data/" + file, encoding="latin-1"03)
#     if file[0] in ['1'] and file != "01.csv":
#         print("reading " + file)
#         df = pd.read_csv("../data/" + file, encoding="latin-1")
#         print("Is BbMxD in? {}".format('BbMxD' in df.columns))
#         print("Is BbMxA in? {}".format('BbMxA' in df.columns))
#         print("Is BbMxH in? {}".format('BbMxH' in df.columns))

01.csv
02.csv
03.csv
04.csv


ParserError: Error tokenizing data. C error: Expected 57 fields in line 305, saw 72
